In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm


DATADIR = 'videos'
CATEGORIES = ['Fire', 'NoFire']

IMG_SIZE = 128
def create_training_data():
    training_data = []
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
              
    return training_data


In [5]:
training_data = create_training_data()

100%|██████████| 1301/1301 [00:19<00:00, 65.75it/s]


In [6]:
import random

print(len(training_data))
random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

2423
0
1
1
0
0
1
0
1
1
1


In [7]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0
X.shape[1:]
Y = np.array(Y)

In [11]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

model = Sequential()


model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=2, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
history = model.fit(X, Y, batch_size=32, epochs=100,validation_split=0.3)

Epoch 1/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 104ms/step - accuracy: 0.5619 - loss: 0.7216 - val_accuracy: 0.7469 - val_loss: 0.5586
Epoch 2/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.7522 - loss: 0.5587 - val_accuracy: 0.7717 - val_loss: 0.5129
Epoch 3/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.7716 - loss: 0.4784 - val_accuracy: 0.7840 - val_loss: 0.5274
Epoch 4/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step - accuracy: 0.8004 - loss: 0.4770 - val_accuracy: 0.7840 - val_loss: 0.4710
Epoch 5/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.8102 - loss: 0.4325 - val_accuracy: 0.7923 - val_loss: 0.4317
Epoch 6/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 103ms/step - accuracy: 0.8234 - loss: 0.4062 - val_accuracy: 0.8267 - val_loss: 0.3853
Epoch 7/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step - accuracy: 0.8475 - loss: 0.3502 - val_accuracy: 0.8157 - val_loss: 0.4149
Epoch 8/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.8524 - loss: 0.3483 - val_accurac

KeyboardInterrupt: 

In [ ]:
hdf5_model_path = 'fire_detection.h5'
model.save(hdf5_model_path, save_format='h5')

ValueError: File format not supported: filepath=fire_detection.model. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(fire_detection.model, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).